In [ ]:
import ray
import ray.data
import pandas as pd

In [ ]:
from hdfs import Config

client = Config(path="./config/.hdfscli.cfg").get_client(
    "dev"
)

In [ ]:
# Insert files, taken from populate_hdfs
files_to_upload = ["transfers.csv","competitions.csv", "appearances.csv", "clubs.csv", "games.csv", "players.csv"]

remote_path = "/data/"

In [ ]:
# make sure remote path exists
client.makedirs(remote_path)

# Insert files, taken from populate_hdfs

# Check if the file exists
for file in files_to_upload:
    local_path = f"./data/{file}"
    print(f"Checking if {file} exists in {remote_path}...")
    if client.status(remote_path + file, strict=False):
        print(f"{file} exists in {remote_path}!")
        continue

    print(f"{file} does not exist in {remote_path}!")
    print(f"Uploading {file} to {remote_path}...")
    # Upload a file to tmp, to be processed further
    client.upload(remote_path, local_path)

print(f"contents in {remote_path}: ", client.list("/data"))

## Ray

In [ ]:
# Initialize Ray
ray.init(dashboard_host="0.0.0.0")

In [ ]:
# Read files

from io import StringIO, BytesIO

# Helper function to read CSV files from HDFS in chunks
def read_csv_from_hdfs_in_chunks(client, file_path, chunksize=100000):
    with client.read(file_path) as reader:
        file_contents = reader.read()
    
    buffer = BytesIO(file_contents)
    chunks = pd.read_csv(buffer, chunksize=chunksize)
    return ray.data.from_pandas_refs([ray.put(chunk) for chunk in chunks])

# Read CSV files
transfers_ds = read_csv_from_hdfs_in_chunks(client, "/data/transfers.csv")
clubs_ds = read_csv_from_hdfs_in_chunks(client, "/data/clubs.csv")
competitions_ds = read_csv_from_hdfs_in_chunks(client, "/data/competitions.csv")

### Pre-processing

1. Remove entries where `transfer_fee == NaN`, since these entries are usually internal transfers (or from lower league youth teams).
2. Filter entries where `market_value_in_eur == Nan`, since we assume it's hard to find any info about these players

For now, we already execute/apply the filtering. But in the future, we will do all the processing first and then train our model on the batches, (hopefully) never applying `take_all`.

In [ ]:
# Filter out rows with null values in 'transfer_fee' and 'market_value_in_eur'
def filter_transfers(batch):
    return batch[
        batch['transfer_fee'].notna() & 
        (~batch['transfer_fee'].isna()) & 
        batch['market_value_in_eur'].notna() & 
        (~batch['market_value_in_eur'].isna())
    ]

transfers_ds = transfers_ds.map_batches(filter_transfers, batch_format="pandas")

# Prepare clubs dataset
clubs_df = clubs_ds.select_columns(['club_id', 'domestic_competition_id']).to_pandas()

# Helper function for merging with clubs
def merge_transfers_clubs(transfers_batch):
    transfers_df = pd.DataFrame(transfers_batch)
    
    merged_from = transfers_df.merge(clubs_df, left_on='from_club_id', right_on='club_id', how='left', suffixes=('', '_from'))
    merged_from = merged_from.rename(columns={'domestic_competition_id': 'from_competition_id'})
    
    merged_to = merged_from.merge(clubs_df, left_on='to_club_id', right_on='club_id', how='left', suffixes=('', '_to'))
    merged_to = merged_to.rename(columns={'domestic_competition_id': 'to_competition_id'})
    
    return merged_to.drop(columns=['club_id', 'club_id_to', 'transfer_date'])

# Merge transfers with clubs
transfers_ds = transfers_ds.map_batches(merge_transfers_clubs, batch_format="pandas")

# Filter out null competition IDs
transfers_ds = transfers_ds.filter(lambda row: row['from_competition_id'] is not None and row['to_competition_id'] is not None)

# Prepare competitions dataset
competitions_df = competitions_ds.select_columns(['competition_id', 'country_name', 'sub_type']).to_pandas()

# Helper function for merging with competitions
def merge_transfers_competitions(transfers_batch):
    transfers_df = pd.DataFrame(transfers_batch)
    
    merged_from = transfers_df.merge(competitions_df, left_on='from_competition_id', right_on='competition_id', how='left', suffixes=('', '_from'))
    merged_from = merged_from.rename(columns={'country_name': 'from_country_name', 'sub_type': 'from_sub_type'})
    
    merged_to = merged_from.merge(competitions_df, left_on='to_competition_id', right_on='competition_id', how='left', suffixes=('', '_to'))
    merged_to = merged_to.rename(columns={'country_name': 'to_country_name', 'sub_type': 'to_sub_type'})
    
    return merged_to.drop(columns=['competition_id', 'competition_id_to'])

# Merge transfers with competitions
transfers_ds = transfers_ds.map_batches(merge_transfers_competitions, batch_format="pandas")

# Display the first few rows
print(transfers_ds.take(5))

# If you need the final result as a Pandas DataFrame:
# final_df = transfers_ds.to_pandas()

## Pandas

In [ ]:
# Load the data
from io import StringIO

with client.read("/data/transfers.csv") as reader:
    file_contents = reader.read().decode('utf-8')

transfers_df = pd.read_csv(StringIO(file_contents))

with client.read("/data/clubs.csv") as reader:
    file_contents = reader.read().decode('utf-8')

clubs_df = pd.read_csv(StringIO(file_contents))

with client.read("/data/competitions.csv") as reader:
    file_contents = reader.read().decode('utf-8')

competitions_df = pd.read_csv(StringIO(file_contents))

with client.read("/data/players.csv") as reader:
    file_contents = reader.read().decode('utf-8')

players_df = pd.read_csv(StringIO(file_contents))

In [ ]:
# Filter transfers
# 1. Remove entries where `transfer_fee == NaN`, since these entries are usually internal transfers (or from lower league youth teams).
# 2. Filter entries where `market_value_in_eur == Nan`, since we assume it's hard to find any info about these players

transfers_df = transfers_df[transfers_df['transfer_fee'].notna()]
transfers_df = transfers_df[transfers_df['market_value_in_eur'].notna()]

In [ ]:
# Join tables
transfers_df = transfers_df.merge(clubs_df[['club_id', 'domestic_competition_id']], left_on='from_club_id', right_on='club_id', how='left', validate='m:m')
transfers_df = transfers_df.rename(columns={'domestic_competition_id': 'from_competition_id'})

transfers_df = transfers_df.merge(clubs_df[['club_id', 'domestic_competition_id']], left_on='to_club_id', right_on='club_id', how='left', validate='m:m')
transfers_df = transfers_df.rename(columns={'domestic_competition_id': 'to_competition_id'})

transfers_df = transfers_df.drop(columns=['club_id_x', 'club_id_y', 'transfer_date'])
transfers_df = transfers_df.dropna(subset=['from_competition_id', 'to_competition_id'])

transfers_df = transfers_df.merge(competitions_df[['competition_id', 'country_name', 'sub_type']], left_on='from_competition_id', right_on='competition_id', how='left', validate='m:m')
transfers_df = transfers_df.rename(columns={'country_name': 'from_country_name', 'sub_type': 'from_sub_type'})

transfers_df = transfers_df.merge(competitions_df[['competition_id','country_name', 'sub_type']], left_on='to_competition_id', right_on='competition_id', how='left', validate='m:m')
transfers_df = transfers_df.rename(columns={'country_name': 'to_country_name', 'sub_type': 'to_sub_type'})

transfers_df = transfers_df.drop(columns=['competition_id_x', 'competition_id_y'])

(Potential additional steps)

3. Remove retired players
4. Drop `transfer_date` column, as we don't need it for anything (the `transfer_season` should be enough for everything time-related).
5. Drop one of `from_club_name` or `from_club_id` (and the same for `to_club_...`).

In [ ]:
# Join players_df to transfers_df (columns last_season, country_of_birth, position, sub_position, contract_expiration_date, highest_market_value_in_eur)
# using the player_id
transfers_df = transfers_df.merge(players_df[['player_id', 'last_season', 'country_of_citizenship', 'position', 'sub_position', 'contract_expiration_date', 'highest_market_value_in_eur']], on='player_id', how='left', validate='m:m')

# Filter 'retired' players 
transfers_df = transfers_df[transfers_df['last_season'] > 2023]

transfers_df

In [ ]:
# Other useful tables and their attributes

# appearances.csv - minutes played, goals, assists
# (Would be hard to map to individual players playing, e.g. how do we know who was on the pitch when a goal was scored or conceded?) 
# club_games.csv - own_position, opponent_goals, opponent_position
# clubs.csv - domestic_competition_id, squad_size, average_age, foreigners_percentage, national_team_players, net_transfer_record, (maybe to filter outdated clubs) last_season
# (IMO useless) competitions.csv
# game_events.csv - player_id, type (goal, assist, card)
# (To know no. of games started) game_lineups.csv - player_id, position, type (substitute, starter)
# (IMO useless) games.csv
# (Useful for training, to know the valuation at the time of transfer, maybe 1 year prior?) player_valuations.csv - date, market_value_in_eur, current_club_id, player_id
# players.csv - last_season (filter retired players), country_of_birth, country_of_citizenship, position, sub_position, foot, height_in_cm, contract_expiration_date, agent_name, market_value_in_eur, highest_market_value_in_eur

## Variables (This description is outdated)

Independent - player information (from other tables), `market_value_in_eur`, `from_club_name`/`from_club_id`

Dependant - `to_club_id`/`to_club_name`, `transfer_fee`

Because we have multiple dependant variables, there would be two models - one regression one predicting the transfer fee and another one (classifier most likely) predicting the club ID/name.

When using as a service, it'd be nice if `player_id` and `to_club_name` were only necessary inputs and the rest read from HDFS/other data storage.
Let's presume that in these scenarios, the `transfer_season` would be the current one (24/25).

Representing club names/ids the best way possible:
- initially as IDs, but that could be interpreted as ordinality by the model
- ideally as embeddings - either of the club name or combinations such as "club country + league + club name"

In [ ]:
prepared_transfers_df = transfers_df.copy()
# Replace transfer_season with transfer_season_num
prepared_transfers_df['transfer_season_end_year'] = prepared_transfers_df['transfer_season'].apply(lambda x: int(x.split('/')[0]) + 1)

## Replace from_sub_type and to_sub_type columns from "first_tier", etc. with a number
# All our transfers are first division for some reason
prepared_transfers_df['from_sub_type_num'] = prepared_transfers_df['from_sub_type'].apply(lambda x: {'first_tier': 1, 'second_tier': 2, 'third_tier': 3, 'fourth_tier': 4}.get(x, 0))
prepared_transfers_df['to_sub_type_num'] = prepared_transfers_df['to_sub_type'].apply(lambda x: {'first_tier': 1, 'second_tier': 2, 'third_tier': 3, 'fourth_tier': 4}.get(x, 0))

# Replace countries with IDs for from_country_name, to_country_name, country_of_citizenship columns
def get_country_id_mapping(df, columns):
    all_countries = set()
    for col in columns:
        all_countries.update(df[col].unique())
    return {country: idx for idx, country in enumerate(sorted(all_countries))}

country_columns = ['from_country_name', 'to_country_name', 'country_of_citizenship']
country_id_mapping = get_country_id_mapping(prepared_transfers_df, country_columns)

for col in country_columns:
    prepared_transfers_df[f'{col}_id'] = prepared_transfers_df[col].map(country_id_mapping)
    prepared_transfers_df = prepared_transfers_df.drop(columns=[col])

# Replace position and sub_position respectively using the same approach
def get_position_id_mapping(df):
    all_positions = df['position'].unique()
    return {position: idx for idx, position in enumerate(sorted(all_positions))}

position_mapping = get_position_id_mapping(prepared_transfers_df)

prepared_transfers_df['position_id'] = prepared_transfers_df['position'].map(position_mapping)
prepared_transfers_df = prepared_transfers_df.drop(columns=['position'])

def get_sub_position_id_mapping(df):
    all_sub_positions = df['sub_position'].unique()
    return {sub_position: idx for idx, sub_position in enumerate(sorted(all_sub_positions))}

sub_position_mapping = get_sub_position_id_mapping(prepared_transfers_df)

prepared_transfers_df['sub_position_id'] = prepared_transfers_df['sub_position'].map(sub_position_mapping)
prepared_transfers_df = prepared_transfers_df.drop(columns=['sub_position'])

# Convert contract_expiration_date
prepared_transfers_df['contract_expiration_date'] = pd.to_datetime(prepared_transfers_df['contract_expiration_date'])
prepared_transfers_df['contract_expiration_date'] = prepared_transfers_df['contract_expiration_date'].dt.year

train_features = ['player_id', 'from_club_id', 'market_value_in_eur', 
                  'transfer_season_end_year', 'from_sub_type_num', 'from_country_name_id', 
                  'country_of_citizenship_id', 'position_id', 'sub_position_id', 
                  'contract_expiration_date', 'highest_market_value_in_eur']

X = prepared_transfers_df[train_features]
y = prepared_transfers_df['to_club_id']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
y_pred_df = pd.DataFrame({'player_id': X_test['player_id'], 'predicted_club_id': y_pred})

# Join y_pred_df with players_df on player_id
y_pred_df = y_pred_df.merge(players_df[['player_id', 'name']], on='player_id', how='left')

# Join y_pred_df with clubs_df on predicted_club_id
y_pred_df = y_pred_df.merge(clubs_df[['club_id', 'name']], left_on='predicted_club_id', right_on='club_id', how='left')

# Rename columns for clarity
y_pred_df = y_pred_df.rename(columns={'name_x': 'player_name', 'name_y': 'predicted_club_name'})

# Print y_pred_df
print(y_pred_df[['player_id', 'player_name', 'predicted_club_id', 'predicted_club_name']].head(10))

In [ ]:
import numpy as np

def predict_transfer_probability(player_data, target_club_id):
    # Ensure player_data has all necessary features
    for feature in train_features:
        if feature not in player_data:
            raise ValueError(f"Missing feature: {feature}")
    
    # Create a 2D array with a single sample
    input_data = np.array([player_data[feature] for feature in train_features]).reshape(1, -1)
    
    # Get probabilities for all classes
    probabilities = clf.predict_proba(input_data)[0]
    
    # Find the index of the target club ID in the classes
    target_index = np.where(clf.classes_ == target_club_id)[0]
    
    # Return the probability for the target club
    if len(target_index) > 0:
        return probabilities[target_index[0]]
    else:
        return 0.0  # Return 0 if the club ID is not in the training data

In [ ]:
# Random Czech player
example_player = X[X['player_id'] == 195778].loc[0]

target_club_ids = [27, 31, 40, 984]  # Example club IDs

for club_id in target_club_ids:
    probability = predict_transfer_probability(example_player, club_id)
    club_name = clubs_df[clubs_df['club_id'] == club_id]['name'].values[0]
    print(f"Probability of transfer to {club_name} (ID: {club_id}): {probability:.2%}")

In [ ]:
# Shutdown Ray
# ray.shutdown()